In [0]:
from pyspark import SparkContext, SparkConf
sc

SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
spdf = spark.read.csv("dbfs:/FileStore/tables/advertising.csv", header=True, mode="DROPMALFORMED")
from pyspark.sql import functions as F
spdf = spdf.select([F.col(x).alias(x.lower()) for x in spdf.columns]) #convert to lower case

for col in spdf.columns:
  spdf = spdf.withColumnRenamed(col,col.replace(" ", "_")) #replace space with comma in column name

In [0]:
print((spdf.count(), len(spdf.columns))) #dimensions of the data

(1000, 10)

In [0]:
# spdf.show()

In [0]:
spdf.where(spdf.clicked_on_ad ==1).groupby(['country']).count().sort(['count'],ascending=False).show(1)

+-------+-----+
country|count|
+-------+-----+
 Turkey| 7|
+-------+-----+
only showing top 1 row

In [0]:
spdf.where(spdf.clicked_on_ad ==1).groupby(['city']).count().sort(['count'],ascending=False).show(1)

+---------+-----+
 city|count|
+---------+-----+
Lisamouth| 2|
+---------+-----+
only showing top 1 row

In [0]:
#  3. Considering the 24 hours day as day (6 AM - 6 PM) and night (6 PM - 6 AM), the ad was mostly clicked during the day time or night time. What are the assumptions are you making in doing this analysis? How would you mitigate this assumption?

from pyspark.sql.functions import to_timestamp
spdf = spdf.withColumn('timestamp_formatted',to_timestamp(spdf.timestamp, 'yyyy-MM-dd HH:mm:ss'))

from pyspark.sql.functions import hour
spdf = spdf.withColumn('hour',hour(spdf.timestamp_formatted))
# spdf.display()

spdf = spdf.withColumn('day_night', F.when((F.col("hour").between(6,18 )), "day").otherwise('night'))
spdf.groupby(['day_night']).count().show()

+---------+-----+
day_night|count|
+---------+-----+
 day| 535|
 night| 465|
+---------+-----+

In [0]:
spdf = spdf.withColumn("area_income", spdf.area_income.cast('float'))
spdf.groupby(['city','country']).sum('area_income').sort(['sum(area_income)'],ascending=False).show(1)

+-----------+----------+----------------+
 city| country|sum(area_income)|
+-----------+----------+----------------+
Edwardmouth|Madagascar| 79484.796875|
+-----------+----------+----------------+
only showing top 1 row

In [0]:
spdf = spdf.withColumn("daily_time_spent_on_site", spdf.daily_time_spent_on_site.cast('float'))
spdf.groupby(['age']).sum('daily_time_spent_on_site').withColumnRenamed('sum(daily_time_spent_on_site)','avg_time_spent').\
sort('avg_time_spent',ascending=False).show(1)

+---+-----------------+
age| avg_time_spent|
+---+-----------------+
 31|4263.920009613037|
+---+-----------------+
only showing top 1 row

In [0]:
# Could you create a model for predicting ad click? Create a training data and test data. Present what is your prediction error.